In [20]:
import numpy as np
from pyCoCo import pyCoCo
import sqlite3

In [2]:
fltPath = b"/Users/szymon/Projects/CoCo/data/filters"
rootPath = b"/Users/szymon/Projects/CoCo"

In [9]:
%%timeit 

coco = pyCoCo(fltPath, rootPath)

1 loop, best of 3: 2.15 s per loop


In [3]:
coco = pyCoCo(fltPath, rootPath)

In [13]:
mjd = np.arange(55000, 55100, 2.5, dtype=float)
flt = [b"SDSS_g", b"SDSS_g", b"SDSS_g", b"SDSS_g", b"SDSS_g",
       b"SDSS_g", b"SDSS_g", b"SDSS_g", b"SDSS_g", b"SDSS_g",
       b"SDSS_z", b"SDSS_z", b"SDSS_z", b"SDSS_z", b"SDSS_z",
       b"SDSS_z", b"SDSS_z", b"SDSS_z", b"SDSS_z", b"SDSS_z",
       b"SDSS_r", b"SDSS_r", b"SDSS_r", b"SDSS_r", b"SDSS_r",
       b"SDSS_r", b"SDSS_r", b"SDSS_r", b"SDSS_r", b"SDSS_r",
       b"SDSS_i", b"SDSS_i", b"SDSS_i", b"SDSS_i", b"SDSS_i",
       b"SDSS_i", b"SDSS_i", b"SDSS_i", b"SDSS_i", b"SDSS_i"]
flt = np.array(flt)

In [15]:
%%timeit

coco.simulate(b"SN2011dh", 0.01, -1.0, 0.2, 0.1, 3.1, 55000.0, mjd, flt)

10 loops, best of 3: 75.9 ms per loop


In [35]:
out = coco.simulate(b"SN2011dh", 0.1, -1.0, 0.2, 0.1, 3.1, 55000.0, mjd, flt)
out

array([[  1.33290560e-15,   1.32729091e-15,   1.31267674e-15,
          1.29030705e-15,   1.26136866e-15,   1.22697461e-15,
          1.18815376e-15,   1.14584505e-15,   1.10089552e-15,
          1.05406103e-15,   3.04393698e-15,   2.89662344e-15,
          2.74891504e-15,   2.60211451e-15,   2.45732067e-15,
          2.31544703e-15,   2.17724014e-15,   2.04329729e-15,
          1.91408355e-15,   1.78994762e-15,   1.36044258e-15,
          1.26818549e-15,   1.18046154e-15,   1.09728700e-15,
          1.01863431e-15,   9.44439513e-16,   8.74608824e-16,
          8.09024480e-16,   7.47549834e-16,   6.90033812e-16,
          1.40598913e-15,   1.29530908e-15,   1.19226258e-15,
          1.09646219e-15,   1.00752041e-15,   9.25053376e-16,
          8.48683869e-16,   7.78043753e-16,   7.12775889e-16,
          6.52535615e-16],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,

In [105]:
# db = sqlite3.connect(':memory:')
db = sqlite3.connect('test.db')

In [106]:
conn = db.cursor()
conn.execute(
"""
CREATE TABLE fakes(mjd REAL, flux REAL, fluxerr REAL, filter TEXT)
""")

db.commit()

In [107]:
%%timeit

data = np.zeros(out.shape[1] , dtype=[('mjd', np.float64), ('flux', np.float64), ('fluxerr', np.float64), ('filter', ('U', 10))])
data['mjd'] = mjd
data['flux'] = out[0]
data['fluxerr'] = out[1]
data['filter'] = flt

conn.executemany("""INSERT INTO fakes VALUES (?, ?, ?, ?)""", map(tuple, data.tolist()))
db.commit()

1000 loops, best of 3: 998 µs per loop


In [95]:
conn.execute(
"""
DROP TABLE fakes
""")

db.commit()

In [104]:
db.close()